In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

%matplotlib inline

In [2]:
def DistPlots(data, columns):
    for column in columns:
        sns.distplot(data[column])
        plt.xlabel('value')
        plt.ylabel('density')
        plt.title('Histogram of ' + column)
        plt.show()

In [3]:
data = pd.read_csv('datasets/test_values.csv', index_col=0)
print(data.shape)
data.head()

(500000, 21)


,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,...,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant
row_id,,,,,,,,,,,,,,,,,,,,,
0,2,1,3,1,115.0,3,101,16,276,2,...,1,NaN,6329.0,59.536,69889.0,85.78,1874.0,2410.0,3791,True
1,1,1,1,1,252.0,2,87,20,68,2,...,1,107.0,2473.0,8.050,65313.0,100.00,947.0,1214.0,2839,True
2,1,1,1,1,270.0,1,-1,-1,-1,2,...,2,119.0,NaN,NaN,NaN,NaN,NaN,NaN,4701,False
3,2,1,1,1,179.0,2,376,20,11,2,...,2,44.0,4795.0,29.676,57766.0,100.00,1426.0,1765.0,2153,True
4,2,1,1,1,36.0,2,254,48,156,3,...,3,32.0,5246.0,5.110,63332.0,100.00,1452.0,2092.0,5710,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 21 columns):
loan_type                         500000 non-null int64
property_type                     500000 non-null int64
loan_purpose                      500000 non-null int64
occupancy                         500000 non-null int64
loan_amount                       500000 non-null float64
preapproval                       500000 non-null int64
msa_md                            500000 non-null int64
state_code                        500000 non-null int64
county_code                       500000 non-null int64
applicant_ethnicity               500000 non-null int64
applicant_race                    500000 non-null int64
applicant_sex                     500000 non-null int64
applicant_income                  459859 non-null float64
population                        477520 non-null float64
minority_population_pct           477518 non-null float64
ffiecmedian_family_income         477547 

In [6]:
for col in data.columns:
    if data[col].dtype != object:
        count = 0
        count = [count + 1 for x in data[col] if x == -1]
        print (col + ' ' + str(sum(count)))

loan_type 0
property_type 0
loan_purpose 0
occupancy 0
loan_amount 0
preapproval 0
msa_md 76993
state_code 19120
county_code 20566
applicant_ethnicity 0
applicant_race 0
applicant_sex 0
applicant_income 0
population 0
minority_population_pct 0
ffiecmedian_family_income 0
tract_to_msa_md_income_pct 0
number_of_owner-occupied_units 0
number_of_1_to_4_family_units 0
lender 0
co_applicant 0


Let's clearly define which columns are categorical features and which ones are numerical. There is only a single boolean feature - co_applicant column

In [7]:
cols_numerical = ['loan_amount', 'applicant_income', 'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
                  'ffiecmedian_family_income', 'minority_population_pct', 'tract_to_msa_md_income_pct','population']
cols_categorical = ['loan_type', 'property_type', 'loan_purpose', 'occupancy', 'preapproval', 'msa_md', 'state_code', 
                    'county_code', 'applicant_ethnicity', 'applicant_race', 'applicant_sex', 'lender']

Next, the columns with categorical features should be converted to datatype 'category'.

In [8]:
data[cols_categorical] = data[cols_categorical].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 21 columns):
loan_type                         500000 non-null category
property_type                     500000 non-null category
loan_purpose                      500000 non-null category
occupancy                         500000 non-null category
loan_amount                       500000 non-null float64
preapproval                       500000 non-null category
msa_md                            500000 non-null category
state_code                        500000 non-null category
county_code                       500000 non-null category
applicant_ethnicity               500000 non-null category
applicant_race                    500000 non-null category
applicant_sex                     500000 non-null category
applicant_income                  459859 non-null float64
population                        477520 non-null float64
minority_population_pct           477518 non-null float64
ffiecmed

In [9]:
cols_missing_values = cols_numerical.copy()
cols_missing_values.remove('loan_amount')

data[cols_missing_values].isna().sum()

applicant_income                  40141
number_of_owner-occupied_units    22574
number_of_1_to_4_family_units     22550
ffiecmedian_family_income         22453
minority_population_pct           22482
tract_to_msa_md_income_pct        22517
population                        22480
dtype: int64

In [10]:
rows_with_na = data[data[cols_missing_values].isna().any(axis=1)]
rows_with_na[['number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
             'ffiecmedian_family_income', 'minority_population_pct', 
              'tract_to_msa_md_income_pct','population']].isna().all(axis=1).sum()

22453

Look at the rows in the dataset which have missing numerical values and merge it with the labels column to investigate how critical are the missing values.
Overall, there are 22440 rows of data where the complete set of cenzus data are missing. As this is very close to the individial counts of the missing data, there is probably additional value to include a boolean feature indicating that all data is missing.

In [11]:
data['missing_census'] = data[['number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
                               'ffiecmedian_family_income', 'minority_population_pct', 
                               'tract_to_msa_md_income_pct','population']].isna().all(axis=1)

In [12]:
data['missing_census'].value_counts()

False    477547
True      22453
Name: missing_census, dtype: int64

In [13]:
data.head(30)
data.to_csv('output/test_data_with_missing_census.csv')

In [17]:
rows_with_na = rows_with_na.merge(labels, on='row_id')
rows_with_na.head(30)

,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
row_id,,,,,,,,,,,,,,,,,,,,,
12,2,1,3,1,138.0,3,61,28,3,1,...,NaN,9958.0,41.404,60081.0,100.000,2220.0,2727.0,4664,False,1
16,1,1,3,2,177.0,3,333,6,80,1,...,NaN,6420.0,29.818,68065.0,100.000,1553.0,2001.0,3354,False,0
17,2,2,3,1,138.0,3,-1,37,59,2,...,NaN,4193.0,14.996,57774.0,74.411,1247.0,1998.0,2566,True,1
22,1,1,3,2,20.0,3,109,14,131,2,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4347,False,0
26,1,1,1,1,113.0,1,-1,-1,-1,2,...,54.0,NaN,NaN,NaN,NaN,NaN,NaN,2839,False,0
38,1,1,1,2,88.0,1,-1,-1,-1,2,...,104.0,NaN,NaN,NaN,NaN,NaN,NaN,788,True,0
43,1,1,3,1,214.0,3,274,45,165,2,...,NaN,6623.0,23.543,105446.0,95.496,1763.0,1806.0,209,True,0
45,1,2,1,1,106.0,3,-1,42,136,3,...,48.0,NaN,NaN,NaN,NaN,NaN,NaN,2318,True,0
50,2,1,3,1,193.0,3,109,14,98,2,...,NaN,4967.0,35.597,67570.0,100.000,858.0,1249.0,2240,True,1


From the data above it seems that applicant_income data is missing, the other numerical features are present and vice versa. It also looks as if the missing numerical features have the label of "not accepted". So let's evaluate that theory.

First look at the situation when all the numerical features are missing:

In [18]:
rows_with_na[rows_with_na[['applicant_income', 'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
             'ffiecmedian_family_income', 'minority_population_pct', 
              'tract_to_msa_md_income_pct','population']].isna().all(axis=1)]['accepted'].value_counts()

0    736
1     72
Name: accepted, dtype: int64

Even in case of all numerical information missing, there are still accepted and rejected loans, however there is a strong bias towards not acceptance.

Let's look at the situation when only 'applicant_income' information is missing.

In [19]:
rows_with_na[rows_with_na['applicant_income'].isna()]['accepted'].value_counts()

1    21175
0    18773
Name: accepted, dtype: int64

Even with applicant income missing, the number of accepted and rejected loan applications is very similiar. In fact there is more applications accepted with the income information missing than rejected.

Let's look at the situation when applicant income is not missing, but other numerical features are not present:

In [20]:
rows_with_na[(rows_with_na[['number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
             'ffiecmedian_family_income', 'minority_population_pct', 
              'tract_to_msa_md_income_pct','population']].isna().all(axis=1)) &
             (rows_with_na['applicant_income'].notna())]['accepted'].value_counts()

0    20808
1      824
Name: accepted, dtype: int64

In this case, most data show that the application has not been accepted in the data.

In [21]:
rows_with_na[(rows_with_na[['number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
             'ffiecmedian_family_income', 'minority_population_pct', 
              'tract_to_msa_md_income_pct','population']].isna().all(axis=1)) &
             (rows_with_na['applicant_income'].notna())].head(20)

,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
row_id,,,,,,,,,,,,,,,,,,,,,
22,1,1,3,2,20.0,3,109,14,131,2,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,4347,False,0
26,1,1,1,1,113.0,1,-1,-1,-1,2,...,54.0,NaN,NaN,NaN,NaN,NaN,NaN,2839,False,0
38,1,1,1,2,88.0,1,-1,-1,-1,2,...,104.0,NaN,NaN,NaN,NaN,NaN,NaN,788,True,0
45,1,2,1,1,106.0,3,-1,42,136,3,...,48.0,NaN,NaN,NaN,NaN,NaN,NaN,2318,True,0
58,1,1,1,1,68.0,3,-1,48,-1,2,...,19.0,NaN,NaN,NaN,NaN,NaN,NaN,2654,False,0
79,2,1,1,1,296.0,1,-1,-1,-1,3,...,42.0,NaN,NaN,NaN,NaN,NaN,NaN,3060,False,0
90,2,1,1,1,64.0,1,-1,-1,-1,2,...,42.0,NaN,NaN,NaN,NaN,NaN,NaN,5415,True,0
99,1,1,1,1,139.0,1,-1,-1,-1,3,...,96.0,NaN,NaN,NaN,NaN,NaN,NaN,6349,False,0
120,1,2,1,1,410.0,3,-1,-1,-1,2,...,78.0,NaN,NaN,NaN,NaN,NaN,NaN,4390,False,0
